In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

/tmp/ipykernel_345457/1349790808.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
import pandas as pd

train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')


In [6]:
train.head()

,area,perimeter,major_axis,minor_axis,eccentricity,eqdiasq,solidity,convex_area,extent,aspect_ratio,roundness,compactness,shapefactor_1,shapefactor_2,shapefactor_3,shapefactor_4,target
0,75516,1731.4840,411.7352,245.7620,0.8023,310.0806,0.9148,82546,0.7169,1.6753,0.3165,0.7531,0.0055,0.0033,0.5672,0.9502,1
1,98903,1374.4370,477.2451,269.7676,0.8249,354.8622,0.9585,103181,0.7679,1.7691,0.6579,0.7436,0.0048,0.0027,0.5529,0.9781,0
2,84746,1311.1570,482.7735,235.9040,0.8725,328.4843,0.9121,92914,0.7162,2.0465,0.6195,0.6804,0.0057,0.0028,0.4630,0.9474,1
3,98184,1463.1680,434.3769,292.6472,0.7390,353.5700,0.9543,102890,0.7316,1.4843,0.5763,0.8140,0.0044,0.0030,0.6625,0.9834,0
4,94170,1267.7271,440.1109,278.4162,0.7745,346.2672,0.9643,97656,0.6836,1.5808,0.7363,0.7868,0.0047,0.0030,0.6190,0.9785,0


In [7]:

df = pd.read_excel('Pistachio_28_Features_Dataset.xlsx')



In [8]:
df.head()

,Area,Perimeter,Major_Axis,Minor_Axis,Eccentricity,Eqdiasq,Solidity,Convex_Area,Extent,Aspect_Ratio,...,StdDev_RR,StdDev_RG,StdDev_RB,Skew_RR,Skew_RG,Skew_RB,Kurtosis_RR,Kurtosis_RG,Kurtosis_RB,Class
0,63391,1568.405,390.3396,236.7461,0.7951,284.0984,0.8665,73160,0.6394,1.6488,...,17.7206,19.6024,21.1342,0.4581,0.6635,0.7591,2.9692,3.0576,2.9542,Kirmizi_Pistachio
1,68358,1942.187,410.8594,234.7525,0.8207,295.0188,0.8765,77991,0.6772,1.7502,...,26.7061,27.2112,25.1035,-0.3847,-0.2713,-0.2927,1.9807,2.1006,2.2152,Kirmizi_Pistachio
2,73589,1246.538,452.3630,220.5547,0.8731,306.0987,0.9172,80234,0.7127,2.0510,...,19.0129,20.0703,20.7006,-0.6014,-0.4500,0.2998,3.5420,3.6856,4.1012,Kirmizi_Pistachio
3,71106,1445.261,429.5291,216.0765,0.8643,300.8903,0.9589,74153,0.7028,1.9879,...,18.1773,18.7152,29.7883,-0.6943,-0.6278,-0.7798,2.8776,2.8748,2.8953,Kirmizi_Pistachio
4,80087,1251.524,469.3783,220.9344,0.8823,319.3273,0.9657,82929,0.7459,2.1245,...,23.4298,24.0878,23.1157,-0.9287,-0.8134,-0.4970,2.9915,2.8813,2.7362,Kirmizi_Pistachio


In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold

# === ЗАГРУЗКА ===
train = pd.read_csv("train.csv")
pistachio = pd.read_excel('Pistachio_28_Features_Dataset.xlsx')  # замените на имя файла

# === ОБРАБОТКА ФИСТАШЕК ===
# Приведение к нижнему регистру и совпадающим названиям
rename_dict = {
    'Area': 'area',
    'Perimeter': 'perimeter',
    'Major_Axis': 'major_axis',
    'Minor_Axis': 'minor_axis',
    'Eccentricity': 'eccentricity',
    'Eqdiasq': 'eqdiasq',
    'Solidity': 'solidity',
    'Convex_Area': 'convex_area',
    'Extent': 'extent',
    'Aspect_Ratio': 'aspect_ratio',
    'Roundness' : 'roundness',
    'Compactness' : 'compactness',
    'Shapefactor_1' : 'shapefactor_1',
    'Shapefactor_2' : 'shapefactor_2',
    'Shapefactor_3' : 'shapefactor_3',
    'Shapefactor_4' : 'shapefactor_4'
}
pistachio = pistachio.rename(columns=rename_dict)

# Классы
pistachio['target'] = pistachio['Class'].map({
    'Kirmizi_Pistachio': 1,
    'Siirt_Pistachio': 0
})

# Убираем ненужные колонки
columns_needed = train.columns.drop('target')
pistachio = pistachio[list(columns_needed) + ['target']]

# === ОБЪЕДИНЕНИЕ ===
full_train = pd.concat([train, pistachio], ignore_index=True)

# === ОБУЧЕНИЕ ===
X = full_train.drop(columns=['target'])
y = full_train['target']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# === K-FOLD ВАЛИДАЦИЯ ===
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_scaled, y)):
    print(f"\n=== Fold {fold+1} ===")
    X_train_fold, X_val_fold = X_scaled[train_idx], X_scaled[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

    model = CatBoostClassifier(iterations=1500, learning_rate=0.03, depth=8, verbose=0)
    model.fit(X_train_fold, y_train_fold)

    val_preds = model.predict(X_val_fold)
    acc = accuracy_score(y_val_fold, val_preds)
    print(f"Validation Accuracy: {acc:.4f}")
    accuracies.append(acc)

print(f"\n📊 Average K-Fold Accuracy: {np.mean(accuracies):.4f}")

# === ОБУЧЕНИЕ НА ВСЕМ ДАТАСЕТЕ ===
model.fit(X_scaled, y)

# === ПРЕДСКАЗАНИЕ ===
test = pd.read_csv("test.csv")
test_scaled = scaler.transform(test)
preds = model.predict(test_scaled)

# === СОХРАНЕНИЕ ===




=== Fold 1 ===
Validation Accuracy: 0.9593

=== Fold 2 ===
Validation Accuracy: 0.9636

=== Fold 3 ===
Validation Accuracy: 0.9491

=== Fold 4 ===
Validation Accuracy: 0.9578

=== Fold 5 ===
Validation Accuracy: 0.9491

📊 Average K-Fold Accuracy: 0.9558


In [13]:
submission = pd.DataFrame(preds, columns=['target'])
submission.to_csv("answers.csv", index=False)

In [59]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

cv  = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [60]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(
    hidden_layer_sizes=(256,128,64),
    activation='relu',
    solver='adam',
    alpha=1e-4,
    learning_rate_init=1e-3,
    max_iter=300,
    early_stopping=True,
    random_state=42
)

pipe = Pipeline([('prep', preproc), ('clf', mlp)])

scores = cross_val_score(pipe, X, y, cv=cv, scoring='accuracy', n_jobs=-1)
print(f'CV accuracy: {scores.mean():.4f} &plusmn; {scores.std():.4f}')

CV accuracy: 0.8618 &plusmn; 0.0086


In [61]:
# pip install tensorflow tabnet
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_nn(input_dim):
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(64,  activation='relu'),
        layers.Dense(1,   activation='sigmoid')
    ])
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [62]:
pipe.fit(X, y)
y_pred = pipe.predict(test)

pd.Series(y_pred, name='target').to_csv('answers.csv', index=False)

In [64]:
# pip install lightgbm scikit-learn pandas numpy joblib

import pandas as pd
import joblib
from sklearn.model_selection import StratifiedKFold, cross_val_score
from lightgbm             import LGBMClassifier

RANDOM_STATE = 42
N_SPLITS     = 5

# --- 1. данные
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")
X, y  = train.drop(columns=["target"]), train["target"]

# --- 2. модель
lgbm = LGBMClassifier(
    objective="binary",
    n_estimators=1200,        # сколько деревьев
    learning_rate=0.03,
    num_leaves=128,           # &laquo;ширина&raquo; деревьев
    max_depth=-1,             # -1 = без ограничения
    subsample=0.8,            # bagging_freq=1 по-умолчанию
    colsample_bytree=0.8,
    reg_lambda=1.0,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

# --- 3. k-fold валидация
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
scores = cross_val_score(lgbm, X, y, cv=cv, scoring="accuracy", n_jobs=-1)
print(f"LightGBM {N_SPLITS}-fold accuracy: {scores.mean():.4f} &plusmn; {scores.std():.4f}")

# --- 4. обучаемся на всём train &rarr; предсказываем test
lgbm.fit(X, y)
preds = lgbm.predict(test)

pd.Series(preds, name="target").to_csv("answers.csv", index=False)
joblib.dump(lgbm, "drone_lgbm.joblib")

[LightGBM] [Info] Number of positive: 590, number of negative: 441
[LightGBM] [Info] Number of positive: 590, number of negative: 441
[LightGBM] [Info] Number of positive: 590, number of negative: 440
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.087879 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3621
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.101584 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3621
[LightGBM] [Info] Number of data points in the train set: 1031, number of used features: 16
[LightGBM] [Info] Number of data points in the train set: 1031, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.572260 -> initscore=0.291078
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.572260 -> initscore=0.291078
[LightGBM] [Info] Start training from score 0.291078
[LightGBM] [

KeyboardInterrupt: 